In this file I start by implementing a compressed sensing reconstruction approach. I want to have a well accepted classical method 
as a baseline for my later deep learning models.

In [1]:
import sys
import os

sys.path.append('../scripts')
sys.path.append('../models')

os.environ["CUDA_VISIBLE_DEVICES"]= '1' #, this way I would choose GPU 3 to do the work

from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import torch
import numpy as np
import time
import h5py
import matplotlib.pyplot as plt
from scipy.ndimage import zoom # for compressing images / only for testing purposes to speed up NN training
from scipy.fft import fft2, fftshift
from scipy.io import loadmat
from torch.utils.data import DataLoader, Subset
import torch.optim as optim
import torch.nn as nn
from scipy.io import savemat
import nibabel as nib
import h5py

import plotly.graph_objects as go

# Check Masks for nans and infs

In [9]:
for Vol in range(5,10):
    brain_mask = np.load(f'Vol{Vol}/masks/brain_mask.npy')
    lipid_mask = np.load(f'Vol{Vol}/masks/lipid_mask.npy')

    print('Volume')
    print(Vol)
    print('Brainmask:')
    has_nonfinite = ~np.isfinite(brain_mask).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True

    print('LipidMask:')
    has_nonfinite = ~np.isfinite(lipid_mask).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True

Volume
5
Brainmask:
Contains NaN or Inf? False
LipidMask:
Contains NaN or Inf? False
Volume
6
Brainmask:
Contains NaN or Inf? False
LipidMask:
Contains NaN or Inf? False
Volume
7
Brainmask:
Contains NaN or Inf? False
LipidMask:
Contains NaN or Inf? False
Volume
8
Brainmask:
Contains NaN or Inf? False
LipidMask:
Contains NaN or Inf? False
Volume
9
Brainmask:
Contains NaN or Inf? False
LipidMask:
Contains NaN or Inf? False


# check original data for nans and infs

In [10]:
for Vol in range(5,10):

    data = np.load(f'Vol{Vol}/OriginalData/data.npy')
    IsolatedWater = np.load(f'Vol{Vol}/OriginalData/IsolatedWater.npy')
    SupressedWater = np.load(f'Vol{Vol}/OriginalData/SupressedWater.npy')

    print('Volume')
    print(Vol)
    print('data:')
    has_nonfinite = ~np.isfinite(data).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True

    print('IsolatedWater:')
    has_nonfinite = ~np.isfinite(IsolatedWater).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True

    print('SupressedWater:')
    has_nonfinite = ~np.isfinite(SupressedWater).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True

Volume
5
data:
Contains NaN or Inf? False
IsolatedWater:
Contains NaN or Inf? False
SupressedWater:
Contains NaN or Inf? False
Volume
6
data:
Contains NaN or Inf? False
IsolatedWater:
Contains NaN or Inf? False
SupressedWater:
Contains NaN or Inf? False
Volume
7
data:
Contains NaN or Inf? False
IsolatedWater:
Contains NaN or Inf? False
SupressedWater:
Contains NaN or Inf? False
Volume
8
data:
Contains NaN or Inf? False
IsolatedWater:
Contains NaN or Inf? False
SupressedWater:
Contains NaN or Inf? False
Volume
9
data:
Contains NaN or Inf? False
IsolatedWater:
Contains NaN or Inf? False
SupressedWater:
Contains NaN or Inf? False


# Check Simulations for nans and infs

In [3]:
for Vol in range(5,10):
    with h5py.File(f'Vol{Vol}/TrainData/TrainData_v_1.0.h5', 'r') as f:
        # List all top-level groups/datasets
        print('Vol', Vol)
        print("Keys:", list(f.keys()))
        lipid = f['lipid'][:]

        has_nonfinite = ~np.isfinite(lipid).all()
        print("Contains NaN or Inf?", has_nonfinite)  # True

        lipid_proj = f['lipid_proj'][:]

        has_nonfinite = ~np.isfinite(lipid_proj).all()
        print("Contains NaN or Inf?", has_nonfinite)  # True
        
        lipid_projOP = f['lipid_projOP'][:]

        has_nonfinite = ~np.isfinite(lipid_projOP).all()
        print("Contains NaN or Inf?", has_nonfinite)  # True

        metab = f['metab'][:]

        has_nonfinite = ~np.isfinite(metab).all()
        print("Contains NaN or Inf?", has_nonfinite)  # True

        spectra = f['spectra'][:]
        has_nonfinite = ~np.isfinite(spectra).all()
        print("Contains NaN or Inf?", has_nonfinite)  # True

        water = f['water'][:]
        has_nonfinite = ~np.isfinite(water).all()
        print("Contains NaN or Inf?", has_nonfinite)  # True


Vol 5
Keys: ['lipid', 'lipid_proj', 'lipid_projOP', 'metab', 'spectra', 'water']
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Vol 6
Keys: ['lipid', 'lipid_proj', 'lipid_projOP', 'metab', 'spectra', 'water']
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Vol 7
Keys: ['lipid', 'lipid_proj', 'lipid_projOP', 'metab', 'spectra', 'water']
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Vol 8
Keys: ['lipid', 'lipid_proj', 'lipid_projOP', 'metab', 'spectra', 'water']
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Vol 9
Keys: ['lipid', 'lipid

# Data set 9

In [15]:
Vol = 5

brain_mask = np.load(f'Vol{Vol}/masks/brain_mask.npy')
lipid_mask = np.load(f'Vol{Vol}/masks/lipid_mask.npy')

data = np.load(f'Vol{Vol}/OriginalData/data.npy')
IsolatedWater = np.load(f'Vol{Vol}/OriginalData/IsolatedWater.npy')
SupressedWater = np.load(f'Vol{Vol}/OriginalData/SupressedWater.npy')

In [17]:
Vol = 9
with h5py.File(f'Vol{Vol}/TrainData/TrainData_v_1.0.h5', 'r') as f:
    # List all top-level groups/datasets
    print('Vol', Vol)
    print("Keys:", list(f.keys()))
    lipid = f['lipid'][:]

    has_nonfinite = ~np.isfinite(lipid).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True

    lipid_proj = f['lipid_proj'][:]

    has_nonfinite = ~np.isfinite(lipid_proj).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True
    
    lipid_projOP = f['lipid_projOP'][:]

    has_nonfinite = ~np.isfinite(lipid_projOP).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True

    metab = f['metab'][:]

    has_nonfinite = ~np.isfinite(metab).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True

    spectra = f['spectra'][:]
    has_nonfinite = ~np.isfinite(spectra).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True

    water = f['water'][:]
    has_nonfinite = ~np.isfinite(water).all()
    print("Contains NaN or Inf?", has_nonfinite)  # True

Vol 9
Keys: ['lipid', 'lipid_proj', 'lipid_projOP', 'metab', 'spectra', 'water']
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False
Contains NaN or Inf? False


In [23]:
lip = image_rrrf[headmask > 0, :]        # [N, 840]
lipProj = np.matmul(lip, LipidProj_Operator_ff)  # also [N, 840]


NameError: name 'image_rrrf' is not defined